In [10]:
import pandas as pd
import numpy as np

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 
from itertools import combinations
from scipy import stats

### Множественная проверка гипотез


№2

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле: AUCs.txt

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [2]:
data = pd.read_csv('AUCs.txt', sep='\t')
data.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [12]:
w_stat = pd.DataFrame(columns = ['Model 1', 'Model 2', 'W', 'p-value'])
k = 0
for i, j in combinations([1, 2, 3, 4], 2):
    w_stat.loc[k, 'Model 1'], w_stat.loc[k, 'Model 2'] = data.columns[i], data.columns[j]
    w_stat.loc[k, 'W'], w_stat.loc[k, 'p-value'] = stats.wilcoxon(data.iloc[:, i], data.iloc[:, j])
    k += 1
w_stat

,Model 1,Model 2,W,p-value
0,C4.5,C4.5+m,6.5,0.0107571
1,C4.5,C4.5+cf,43,0.861262
2,C4.5,C4.5+m+cf,11,0.0159064
3,C4.5+m,C4.5+cf,17,0.0463327
4,C4.5+m,C4.5+m+cf,22,0.327826
5,C4.5+cf,C4.5+m+cf,10,0.0229091


In [14]:
top_diff = w_stat.sort_values(by='p-value')
top_diff

,Model 1,Model 2,W,p-value
0,C4.5,C4.5+m,6.5,0.0107571
2,C4.5,C4.5+m+cf,11,0.0159064
5,C4.5+cf,C4.5+m+cf,10,0.0229091
3,C4.5+m,C4.5+cf,17,0.0463327
4,C4.5+m,C4.5+m+cf,22,0.327826
1,C4.5,C4.5+cf,43,0.861262


In [19]:
top_diff[top_diff['p-value'] <= 0.05].shape[0]

4

Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [21]:
reject, p_corrected, a1, a2 = multipletests(w_stat['p-value'], 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [22]:
w_stat['p_corrected'] = p_corrected
w_stat['reject'] = reject

In [23]:
w_stat

,Model 1,Model 2,W,p-value,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.0107571,0.0645428,False
1,C4.5,C4.5+cf,43,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11,0.0159064,0.0795322,False
3,C4.5+m,C4.5+cf,17,0.0463327,0.138998,False
4,C4.5+m,C4.5+m+cf,22,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,10,0.0229091,0.0916364,False


In [27]:
w_stat.reject.value_counts()

False    6
Name: reject, dtype: int64

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [28]:
reject, p_corrected, a1, a2 = multipletests(w_stat['p-value'], 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 
w_stat['p_corrected'] = p_corrected
w_stat['reject'] = reject
w_stat

,Model 1,Model 2,W,p-value,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.0107571,0.0458182,True
1,C4.5,C4.5+cf,43,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11,0.0159064,0.0458182,True
3,C4.5+m,C4.5+cf,17,0.0463327,0.0694991,False
4,C4.5+m,C4.5+m+cf,22,0.327826,0.393391,False
5,C4.5+cf,C4.5+m+cf,10,0.0229091,0.0458182,True


In [29]:
w_stat.reject.value_counts()

True     3
False    3
Name: reject, dtype: int64

№1 - все

№2 - 4,5 +  4,5+m

№3 - 4

№4 - m

№5 - 0

№6 - 3

№7 - не корректно

№8 - не все, точно увеличение числа гиперпараметров и видимо забить на множественную проверку